# Inspect data


In [1]:
import sys
import os
import numpy as np
import plotly.graph_objects as go
import json
import matplotlib.pyplot as plt
from pathlib import Path

sys.path.append(os.path.abspath("../../Leaderboard/"))

from leaderboard.agents.geometric_map import GeometricMap
from leaderboard.utils.constants import Constants

from lac.mapping.mapper import Mapper, interpolate_heights, grid_to_points
from lac.mapping.map_utils import get_geometric_score
from lac.utils.plotting import (
    plot_poses,
    plot_3d_points,
    plot_surface,
    plot_heightmaps,
    plot_rock_map,
    plot_rock_maps,
)
from lac.util import load_data, pos_rpy_to_pose

%load_ext autoreload
%autoreload 2

/home/lac/anaconda3/envs/lac/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LAC_BASE_PATH: None


/home/lac/opt/LightGlue/lightglue/lightglue.py:24: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


### Load data


In [2]:
data_path = "../../output/NavAgent/default_run"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Loaded {len(poses)} poses")

Loaded 2699 poses


### Inspect poses


In [3]:
json_data = json.load(open(f"{data_path}/data_log.json"))
vo_poses = []
for frame in json_data["frames"]:
    vo_poses.append(np.array(frame["est_pose"]))

In [4]:
fig = plot_poses(poses, color="black", no_axes=True, name="Ground Truth")
fig = plot_poses(vo_poses, fig=fig, color="orange", no_axes=True, name="VO")
fig.show()

### Inspect maps


In [5]:
ground_truth_map = np.load(Path(data_path) / "Moon_Map_01_4_rep0.dat", allow_pickle=True)
agent_map = np.load(Path(data_path) / "Moon_Map_01_4_rep0_agent.dat", allow_pickle=True)

FileNotFoundError: [Errno 2] No such file or directory: '../../output/NavAgent/default_run/Moon_Map_01_4_rep0.dat'

In [6]:
get_geometric_score(ground_truth_map, agent_map)

NameError: name 'ground_truth_map' is not defined

In [ ]:
plot_heightmaps(ground_truth_map, agent_map)

In [ ]:
plot_rock_maps(ground_truth_map, agent_map)

In [ ]:
plot_rock_map(ground_truth_map)

In [ ]:
plot_rock_map(agent_map)

In [ ]:
ground_truth_map_points = ground_truth_map[:, :, :3].reshape(-1, 3)
agent_map_points = agent_map[:, :, :3].reshape(-1, 3)

fig = plot_3d_points(ground_truth_map_points, color="blue", name="Ground Truth")
fig = plot_3d_points(agent_map_points, fig=fig, color="red", name="Agent Map")
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

# Inspect rock points


In [ ]:
data_path = "../../output/nav_agent"
rock_points = np.load(f"{data_path}/rock_points.npy")

In [ ]:
fig = plot_3d_points(rock_points)
fig.update_layout(height=700, width=1200, scene_aspectmode="data")
fig.show()

In [ ]:
import os

cams_geoms = json.load(open(os.path.expanduser("~/LunarAutonomyChallenge/docs/geometry.json")))[
    "rover"
]["cameras"]
cams_geoms["FrontLeft"]